In [125]:
#%pip install scikit-learn
#%pip install nltk
#%pip install gensim




In [124]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
sys.path.append('../')
from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces



# Load Data

In [79]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [115]:
# Filter out the Spanish data and reindex
df_es = df[df["language"] == "es"]

#Reset index
df_es = df_es.reset_index(drop=True)
df_es

,data,player,language,publishedAt
0,"{'content': ""DIRECTO\nMercado de fichajes de f...",Exequiel Palacios,es,2023-01-29T18:25:03Z
1,{'content': 'Con el primer mes del 2023 a poco...,Exequiel Palacios,es,2023-01-30T16:52:46Z
2,{'content': 'Deportes\nGustavo Puerta ya no ju...,Exequiel Palacios,es,2023-01-31T20:41:38Z
3,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-09T18:32:38Z
4,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-12T21:13:55Z
...,...,...,...,...
226,{'content': 'Piero Hincapié (der.) durante el ...,Piero Hincapié,es,2023-02-16T22:09:36Z
227,{'content': 'Piero Hincapié (centro) y Félix T...,Piero Hincapié,es,2023-02-14T23:40:33Z
228,"{'content': 'El DT de Ecuador Gustavo Alfaro, ...",Piero Hincapié,es,2023-02-14T15:58:38Z
229,{'content': 'Eche un vistazo a la versión beta...,Piero Hincapié,es,2023-02-03T10:58:00Z


In [116]:
print(df_es.loc[0, 'data'])

{'content': "DIRECTO\nMercado de fichajes de fútbol, en directo\nAdeyemi firmó el que es su primer gol en lo que va de Bundesliga\nJavier Alfaro\nRedactor Fútbol\nEl que perdona, lo acaba pagando muy caro en esto del fútbol. En España, en la China y en Alemania. En la liga de este último país lo sufrió este domingo el Bayer Leverkusen de Xabi Alonso, que fue indulgente con el Borussia Dortmund y contra su voluntad acabó cosechando una derrota de esas que se califican como injustas por los méritos realizados. Pero la realidad es que en esto del balompié, justicia a parte, gana el que marca más goles que el rival, y mientras que el Dortmund hizo dos el Leverkusen no logró ninguno (0-2).\nCon Florian Wirtz dejando su lesión definitivamente en el olvido y Exequiel Palacios y Amiri creando juego en la medular, el Bayer Leverkusen llevó la iniciativa en el juego desde el inicio, haciendo latente lo bien trabajando que tiene el equipo el bueno de Xabi Alonso, pero ni los mencionados ni sus co

# Remove duplicates

In [117]:
# Remove the similiar rows (The Function is imported from utils on top)
df_es = remove_similar_rows_per_player(df_es, df_es['player'].unique())

# Returned df is not working well

# Emojis analysis

In [83]:
# Unicode ranges for emojis
emoji_ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0xFE00, 0xFE0F),    # Variation Selectors
    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs
    (0x1F1E6, 0x1F1FF)   # Flags
]

# Function to check if a character is an emoji
def is_emoji(character):
    for start, end in emoji_ranges:
        if ord(character) in range(start, end + 1):
            return True
    return False

def is_emoji_remove(character):
    emoji_ord = ord(character)
    for start, end in emoji_ranges:
        if emoji_ord >= start and emoji_ord <= end:
            return True
    return False

# Assuming your dataframe is named 'df'
articles_with_emoji = 0

# Iterate over the rows of the dataframe
for index, row in df_es.iterrows():
    # Counter for emojis in the current article
    emoji_count = 0

    # Iterate over the columns of the current row
    for column in row.index:
        # Check if the cell contains an emoji
        for char in str(row[column]):
            if is_emoji(char):
                emoji_count += 1

    # If there is at least one emoji in the current article, increment the count of articles with emojis
    if emoji_count > 0:
        articles_with_emoji += 1
        print("Emojis found in article", index + 1, ":", emoji_count)

print("Total number of articles with emojis:", articles_with_emoji)





Emojis found in article 6 : 6
Emojis found in article 12 : 2
Emojis found in article 14 : 4
Emojis found in article 30 : 1
Emojis found in article 32 : 1
Emojis found in article 65 : 4
Emojis found in article 78 : 2
Emojis found in article 81 : 11
Emojis found in article 15 : 11
Emojis found in article 7 : 11
Emojis found in article 11 : 12
Emojis found in article 19 : 11
Emojis found in article 21 : 1
Emojis found in article 3 : 6
Emojis found in article 4 : 3
Emojis found in article 5 : 3
Emojis found in article 9 : 1
Emojis found in article 19 : 15
Emojis found in article 1 : 5
Emojis found in article 21 : 4
Emojis found in article 26 : 20
Emojis found in article 29 : 5
Emojis found in article 30 : 3
Emojis found in article 31 : 2
Total number of articles with emojis: 24


# Remove Patterns

In [107]:
patternlist = [
    r"^{\'content\': \'",
    r"^{\'content\'",
    r"DIRECTO",
    r"Espacio Publicitario",
    r"Copyright.*",
    r"Foto:*",
    r"Todos los derechos reservados:*",
    r"Suscribete*",
   
]

In [136]:
# Create a copy
data_wo_pattern = df_es.copy()
data_wo_pattern.reset_index(drop=True, inplace=True)

#remove patterns
for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))

#link
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

#emoji
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'[^\w\s,]|.', lambda m: '' if is_emoji_remove(m.group()) else m.group(), str(x)))

#remove \n


# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_wo_pattern['data'] = data_wo_pattern['data'].apply(strip_multiple_whitespaces)

# Reset index
data_wo_pattern.reset_index(drop=True, inplace=True)
print(data_wo_pattern.loc[44, 'data'])



BERLÍN (AP) — Amine Adli fue amonestado dos veces por piscinazos, pero ambas amarillas fueron rescindidas en ambas ocasiones, y Bayer Leverkusen anotó dos veces por los penales que se señalaron posteriormente para vencer el domingo 2-1 al Bayern Múnich.\nEl argentino Exequiel Palacios facturó ambos penas máximas para una victoria que dejó al Borussia Dortmund en lo más alto de la Bundesliga.\nEl árbitro Tobias Stieler tuvo que agradecerle al VAR — dos veces — tras equivocarse. Se disculpó con Adli en cada ocasión luego que las revisiones confirmaron que el jugador francés fue víctima de faltas de Benjamin Pavard, en primera instancia, y luego de Dayot Upamecano.\nTras sufrir su tercera derrota en la Bundesliga esta temporada, el Bayern quedó un punto detrás del Dortmund con nueve fechas por delante. El Bayern recibirá al Dortmund el 1 de abril.\n“Era un partido muy importante para nosotros. Lo sabíamos de antemano, pero fuimos incapaces de de mostrarlo durante los 90 minutos y tenemos 

# Find line with player

In [62]:
data_with_playernames = data_wo_pattern.copy()

data_with_playernames = find_lines_with_player(data_with_playernames, data_with_playernames['player'].unique())

In [63]:
print(data_with_playernames.loc[2, 'data'])

2    DeportesnGustavo Puerta ya no jugará en el Bay...
2    nMercado de fichajes de fútbol, en directonFre...
2    Piero Hincapié der durante el partido entre Ba...
2     Piero Hincapié y Exequiel Palacios en el Baye...
2    Piero Hincapié der durante el partido entre Ba...
2    El mercado de invierno en Europa tuvo a un arg...
2    nMercado de fichajes de fútbol, en directonCam...
2    Una de las cancha del complejo de Independient...
Name: data, dtype: object


# Stop words

In [65]:
#Create copy to new df data_rm_sw as in data removed stopwords
data_es_rm = data_with_playernames.copy()

#Load stopwords
#nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')


In [66]:
# Create a list of stop words to remove 
stop_words_to_remove = ['ni', 'no']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  spanish_stopwords.remove(word)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=spanish_stopwords)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_es_rm['data'] = data_es_rm['data'].apply(remove_stopwords_from_text)
data_es_rm['data']


0     nMercado fichajes fútbol, directonAdeyemi firm...
1     Con primer mes 2023 concluir algarabía intacta...
2     DeportesnGustavo Puerta no jugará Bayer Leverk...
3     Dólar bluenAlberto FernándeznMauricio MacrinIN...
4     Dólar bluenAlberto FernándeznMauricio MacrinIN...
                            ...                        
52    Piero Hincapié der partido Bayer Leverkusen Mó...
53    Piero Hincapié centro Félix Torres tratan marc...
54    El DT Ecuador Gustavo Alfaro, partido Países B...
55    Eche vistazo versión beta dwcom Y no Su opinió...
56    El español Andoni Bombín anunciado jueves 2 fe...
Name: data, Length: 229, dtype: object

# More removals

In [70]:
# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_es_rm['data'] = data_es_rm['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_es_rm['data'] = data_es_rm['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_es_rm['data'] = data_es_rm['data'].apply(strip_multiple_whitespaces)

# Convert text to lowercase
data_es_rm['data'] = data_es_rm['data'].str.lower()


# Reset index
data_es_rm.reset_index(drop=True, inplace=True)



In [73]:
print(data_es_rm.loc[3, 'data'])

dólar bluenalberto fernándeznmauricio macrinindecnharvardnboca vs colo colonfin mundonenfrentamiento barrasnfneclipse lunarnhoróscopo hoynnsuscribite nsuscribite ningresarntito lescanona casi dos meses coronación selección argentina mundial qatarxaubde mano lionel messi xala alegría sigue siendo constante integrantes sigschool marca indumentaria haedo diego maradona popularizó utilizar remeras últimas recordadas entrevistas televisivasnla felicidad familia hace marca argentinos tal yair pombar define invención cuenta eslogan barrio va allá nueva conquista copa mundo lionel messi compañía consumaron medio orientenen caso satisfacción ver llegada productos emprendimiento seno mismísima albiceleste es integrantes delegación nacional obtuvo tercera estrella hace tiempo lucen pilcha made in morón caracteriza homenajear honrar ídolos populares personalidades destacadas deporte cultura artenel presidente afa claudio chiqui tapia integrante cuerpo técnico mano derecha dt lionel scaloni pablo a